In [1]:
# Insert code here.
import pandas as pd
import numpy as np
import random
import re
import time
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, BertConfig, AutoModel
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from torch.utils.data import Dataset
from tqdm import tqdm

# from sentence_transformers import SentenceTransformer
# sent_encoder = SentenceTransformer('bert-base-nli-mean-tokens')

In [2]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:1")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
torch.cuda.empty_cache()

There are 4 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [3]:
models = ['bert-base-multilingual-cased', 'xlm-roberta-base', 'sagorsarker/bangla-bert-base', 'ai4bharat/indic-bert']
model_num = 3
tokenizer = AutoTokenizer.from_pretrained(models[model_num])


In [4]:
import pickle
with open('train.pickle','rb') as f:
    train = pickle.load(f)
    train = pd.DataFrame.from_dict(train)
with open('valid.pickle','rb') as f:
    test = pickle.load(f)
    test = pd.DataFrame.from_dict(test)
# test = pd.read_csv('data/valid.tsv', sep='\t')

In [5]:
train.sample(20)

,tweet_id,task_1,full_tweet,tweet_raw_text,hashtags,smiley,emoji,url,mentions,numerals,reserved_word,emotext,segmented_hash,clean
4564,4679,non-hostile,"Poco M2 Pro अब ओपन सेल में उपलब्ध, जानें दाम व...","Poco M2 Pro अब ओपन सेल में उपलब्ध , जानें दाम...",[],[],[],"[https://t.co/dYMuRBgziu, https://t.co/4nqZTgT...",[],[],[],[],[],"Poco M2 Pro अब ओपन सेल में उपलब्ध, जानें दाम व..."
4932,5133,non-hostile,"डॉ. कफील खान पर से हटा NSA, इलाहाबाद HC ने दिय...","डॉ. कफील खान पर से हटा NSA , इलाहाबाद HC ने द...",[],[],[],[https://t.co/W5m2FZjYwj],[],[],[],[],[],"डॉ. कफील खान पर से हटा NSA, इलाहाबाद HC ने दिय..."
768,768,non-hostile,20 साल से जनता के भरोसे पर कायम है आजतक। खबर म...,साल से जनता के भरोसे पर कायम है आजतक। खबर मतलब...,"[#Promo, #AajTakNo1]",[],[],[https://t.co/vc6dVdOZs1],[],[20],[],[],"[promo, aaj tak no 1]",20 साल से जनता के भरोसे पर कायम है आजतक। खबर म...
3160,3275,defamation,"गिरी हुई इकोनॉमी देखकर चाइना सोचेगा, ऐसी गिरी ...","गिरी हुई इकोनॉमी देखकर चाइना सोचेगा , ऐसी गिर...",[],[],[],[],[],[],[],[],[],"गिरी हुई इकोनॉमी देखकर चाइना सोचेगा, ऐसी गिरी ..."
3145,3260,non-hostile,‘राष्ट्रीय भर्ती एजेंसी‘ पर सरकार का निर्णय भर...,‘राष्ट्रीय भर्ती एजेंसी‘ पर सरकार का निर्णय भर...,[#NTA],[],[],"[https://t.co/h1QENG8F5V, https://t.co/I9d22U9...",[@DrJitendraSingh],[],[],[],[nta],‘राष्ट्रीय भर्ती एजेंसी‘ पर सरकार का निर्णय भर...
5394,5595,non-hostile,"कंगना से विवाद के बीच संजय राउत का ट्वीट, 'मेर...","कंगना से विवाद के बीच संजय राउत का ट्वीट , 'म...",[],[],[],[https://t.co/8Do3QWQ7Fr],[],[],[],[],[],"कंगना से विवाद के बीच संजय राउत का ट्वीट, 'मेर..."
835,835,non-hostile,CM ठाकुर ने बताया कि 9 सितंबर को कंगना की मुंब...,CM ठाकुर ने बताया कि सितंबर को कंगना की मुंबई ...,"[#KanganaRanaut, #HimachalPradesh]",[],[],[https://t.co/NHJnhKf8Ba],[@satenderchauhan],[9],[],[],"[kangana ranaut, himachal pradesh]",CM ठाकुर ने बताया कि 9 सितंबर को कंगना की मुंब...
1192,1259,fake,*#जयपुर का #राज-मन्दिर #सिनेमाहॉल बिका...#* *...,*#जयपुर का #राज-मन्दिर #सिनेमाहॉल बिका...#* *...,"[#जयप, #र, #स, #, #प, #, #एश, #, #र, #]",[],[],[],[],"[1, 30]",[],[],"[जयप, र, स, , प, , एश, , र, ]",* ुर का ाज-मन्दिर िनेमाहॉल बिका... * * प्रा...
1019,1019,non-hostile,....से सम्बन्धित परास्नातक के छात्रों को अनुमत...,....से सम्बन्धित परास्नातक के छात्रों को अनुमत...,[],[],[],[],[@AwasthiAwanishK],[],[],[],[],....से सम्बन्धित परास्नातक के छात्रों को अनुमत...
1637,1704,"defamation,hate,offensive",@Samrehman03 .@Samrehman03 बहुत बड़ा कमीना है...,".@Samrehman03 बहुत बड़ा कमीना है ये , सिर्फ ...",[],[],[🙄],[],"[@Samrehman03, @Samrehman03]",[],[],[face with rolling eyes],[],". बहुत बड़ा कमीना है ये, सिर्फ हिन्दू भाइय..."


In [6]:
def encode_labels(label):
    tmp = label.split(',')
    ls = [0, 0, 0, 0]
    if tmp[0] == 'non-hostile':
        return ls
    if 'fake' in tmp:
        ls[0] = 1
    if 'hate' in tmp:
        ls[1] = 1
    if 'offensive' in tmp:
        ls[2] = 1
    if 'defamation' in tmp:
        ls[3] = 1
    return ls

In [7]:
train['encodelabels'] = train['task_1'].apply(encode_labels)

In [8]:
train.sample(20)

,tweet_id,task_1,full_tweet,tweet_raw_text,hashtags,smiley,emoji,url,mentions,numerals,reserved_word,emotext,segmented_hash,clean,encodelabels
719,719,hate,@Sabir51861509 पुलिस आपके लिए नहीं बनी है न्या...,पुलिस आपके लिए नहीं बनी है न्यायपालिका आपके लि...,[],[],[],[],[@Sabir51861509],"[10, 10, 90]",[],[],[],पुलिस आपके लिए नहीं बनी है न्यायपालिका आपके ...,"[0, 1, 0, 0]"
1209,1276,fake,15 जून की रात गलवान घाटी पर हुई उस मुठभेड़। तिर...,जून की रात गलवान घाटी पर हुई उस मुठभेड़। तिरंगे...,[],[],[],[],[],[15],[],[],[],15 जून की रात गलवान घाटी पर हुई उस मुठभेड़। तिर...,"[1, 0, 0, 0]"
2633,2748,non-hostile,पुलिस कस्टडी से बीएचयू के छात्र के लापता होने ...,पुलिस कस्टडी से बीएचयू के छात्र के लापता होने ...,[#Allahabad],[],[],[https://t.co/EpjUpwojjr],[],[],[],[],[allahabad],पुलिस कस्टडी से बीएचयू के छात्र के लापता होने ...,"[0, 0, 0, 0]"
43,43,non-hostile,बधाई हो मोदी जी आपकी मन की बात को पूरे 100 हज़...,बधाई हो मोदी जी आपकी मन की बात को पूरे हज़ार ड...,[],[],[😂],[https://t.co/BUEX28Lfpe],[],[100],[],[face with tears of joy],[],बधाई हो मोदी जी आपकी मन की बात को पूरे 100 हज़...,"[0, 0, 0, 0]"
2860,2975,fake,"नोवल कोरोनावायरस से संक्रमित 20,000 से अधिक रो...","नोवल कोरोनावायरस से संक्रमित 20 , 000 से अधिक ...",[],[],[],[],[],"[20,000]",[],[],[],"नोवल कोरोनावायरस से संक्रमित 20,000 से अधिक रो...","[1, 0, 0, 0]"
805,805,fake,पाकिस्तान में शेष बचे हिन्दुओं के लिए भी कोई म...,पाकिस्तान में शेष बचे हिन्दुओं के लिए भी कोई म...,[],[],[],[],[],[],[],[],[],पाकिस्तान में शेष बचे हिन्दुओं के लिए भी कोई म...,"[1, 0, 0, 0]"
4035,4150,non-hostile,रेल मंत्री @PiyushGoyal ने मुंबई में #NEET_JEE...,रेल मंत्री ने मुंबई में की परीक्षा देने वाले छ...,[#NEET_JEE],[],[],[https://t.co/ly7wl5QP6l],"[@PiyushGoyal, @RailMinIndia]",[],[],[],[neet _ jee],रेल मंत्री ने मुंबई में की परीक्षा देने वा...,"[0, 0, 0, 0]"
5008,5209,non-hostile,विराट कोहली को टीम इंडिया से बाहर करने की आवाज...,विराट कोहली को टीम इंडिया से बाहर करने की आवाज...,[#MSDhoni],[],[],[https://t.co/6T2VP0pf7e],[],[],[],[],[ms dhoni],विराट कोहली को टीम इंडिया से बाहर करने की आवाज...,"[0, 0, 0, 0]"
3247,3362,offensive,हिन्दुत्व की जो नींव मोदीजी ने रखी है. यकीन रख...,हिन्दुत्व की जो नींव मोदीजी ने रखी है. यकीन रख...,[#रण_म],[],"[🚩, 🚩]",[],[],[],[],"[triangular flag, triangular flag]",[रण _म],हिन्दुत्व की जो नींव मोदीजी ने रखी है. यकीन रख...,"[0, 0, 1, 0]"
2340,2455,fake,यह ख़बर जो एजेंट मीडिया नहीं चलाएगा । दक्षिण अम...,यह ख़बर जो एजेंट मीडिया नहीं चलाएगा । दक्षिण अम...,[],[],[],[],[],[],[],[],[],यह ख़बर जो एजेंट मीडिया नहीं चलाएगा । दक्षिण अम...,"[1, 0, 0, 0]"


In [9]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [10]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-05


In [11]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.clean
        self.targets = self.data.encodelabels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_attention_mask = True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [12]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_data=train.sample(frac=train_size,random_state=200)
test_data=train.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (5528, 15)
TRAIN Dataset: (4422, 15)
TEST Dataset: (1106, 15)


In [13]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [14]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained(models[model_num])
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
             

In [15]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [16]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [17]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
#         if _%50==0:
#             print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    print(f'Epoch: {epoch}, Loss:  {loss.item()}')
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    final_outputs = np.array(fin_outputs) >=0.5
    final = []
    final_t = []
    final_fine = [[],[],[],[]]
    final_fine_t = [[],[],[],[]]
    for (i,j) in zip(final_outputs, fin_targets):
        output_sum = sum(i)
        target_sum = sum(j)
        if output_sum == 0:
            final.append(0)
        else:
            final.append(1)
        if target_sum == 0:
            final_t.append(0)
        else:
            final_t.append(1)
        for p in range(4):
            final_fine[p].append(int(i[p]))
            final_fine_t[p].append(int(j[p]))
    print("Coarse:")
    print(classification_report(final, final_t))
    for i in range(4):
        print("Fine", i)
        print(classification_report(final_fine[i], final_fine_t[i]))
#     return fin_outputs, fin_targets

In [18]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]/home/tathagata.raha/miniconda3/envs/fastai/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1764: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
139it [00:32,  4.24it/s]
2it [00:00, 10.95it/s]

Epoch: 0, Loss:  0.3369062840938568


35it [00:03, 10.96it/s]
/home/tathagata.raha/miniconda3/envs/fastai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
0it [00:00, ?it/s]

Coarse:
              precision    recall  f1-score   support

           0       1.00      0.53      0.70      1106
           1       0.00      0.00      0.00         0

    accuracy                           0.53      1106
   macro avg       0.50      0.27      0.35      1106
weighted avg       1.00      0.53      0.70      1106

Fine 0
              precision    recall  f1-score   support

           0       1.00      0.80      0.89      1106
           1       0.00      0.00      0.00         0

    accuracy                           0.80      1106
   macro avg       0.50      0.40      0.44      1106
weighted avg       1.00      0.80      0.89      1106

Fine 1
              precision    recall  f1-score   support

           0       1.00      0.88      0.93      1106
           1       0.00      0.00      0.00         0

    accuracy                           0.88      1106
   macro avg       0.50      0.44      0.47      1106
weighted avg       1.00      0.88      0.93      110

139it [00:33,  4.18it/s]
2it [00:00, 10.76it/s]

...

Coarse:
              precision    recall  f1-score   support

           0       0.95      0.71      0.81       797
           1       0.55      0.91      0.68       309

    accuracy                           0.76      1106
   macro avg       0.75      0.81      0.75      1106
weighted avg       0.84      0.76      0.78      1106

Fine 0
              precision    recall  f1-score   support

           0       0.94      0.89      0.91       933
           1       0.53      0.68      0.59       173

    accuracy                           0.85      1106
   macro avg       0.73      0.78      0.75      1106
weighted avg       0.87      0.85      0.86      1106

Fine 1
              precision    recall  f1-score   support

           0       0.94      0.91      0.93      1002
           1       0.35      0.46      0.40       104

    accuracy                           0.87      1106
   macro avg       0.64      0.69      0.66      1106
weighted avg       0.89      0.87      0.88      110

139it [00:34,  4.06it/s]
2it [00:00, 10.44it/s]

Epoch: 9, Loss:  0.09804540127515793


35it [00:03, 10.55it/s]

Coarse:
              precision    recall  f1-score   support

           0       0.93      0.74      0.82       739
           1       0.63      0.89      0.74       367

    accuracy                           0.79      1106
   macro avg       0.78      0.81      0.78      1106
weighted avg       0.83      0.79      0.80      1106

Fine 0
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       895
           1       0.62      0.65      0.63       211

    accuracy                           0.86      1106
   macro avg       0.77      0.78      0.77      1106
weighted avg       0.86      0.86      0.86      1106

Fine 1
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      1012
           1       0.34      0.50      0.41        94

    accuracy                           0.88      1106
   macro avg       0.65      0.71      0.67      1106
weighted avg       0.90      0.88      0.89      110

In [19]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [20]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

35it [00:03, 10.59it/s]


In [21]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.6835443037974683
Hamming Loss = 0.11867088607594936


In [22]:
from sklearn.metrics import classification_report
total = len(targets)
final = []
final_t = []
final_fine = [[],[],[],[]]
final_fine_t = [[],[],[],[]]
for (i,j) in zip(final_outputs, targets):
    output_sum = sum(i)
    target_sum = sum(j)
    if output_sum == 0:
        final.append(0)
    else:
        final.append(1)
    if target_sum == 0:
        final_t.append(0)
    else:
        final_t.append(1)
    for p in range(4):
        final_fine[p].append(int(i[p]))
        final_fine_t[p].append(int(j[p]))
print("Coarse:")
print(classification_report(final, final_t))
for i in range(4):
    print("Fine", i)
    print(classification_report(final_fine[i], final_fine_t[i]))

Coarse:
              precision    recall  f1-score   support

           0       0.93      0.74      0.82       739
           1       0.63      0.89      0.74       367

    accuracy                           0.79      1106
   macro avg       0.78      0.81      0.78      1106
weighted avg       0.83      0.79      0.80      1106

Fine 0
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       895
           1       0.62      0.65      0.63       211

    accuracy                           0.86      1106
   macro avg       0.77      0.78      0.77      1106
weighted avg       0.86      0.86      0.86      1106

Fine 1
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      1012
           1       0.34      0.50      0.41        94

    accuracy                           0.88      1106
   macro avg       0.65      0.71      0.67      1106
weighted avg       0.90      0.88      0.89      110

In [23]:
acc/total

NameError: name 'acc' is not defined

In [ ]:
final_fine = [[],[],[],[]]

In [ ]:
final_fine